In [22]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

### Activation functions

In [4]:
def relu(x):
    assert isinstance(x, np.ndarray), "Input to ReLU must be a numpy array"
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU output must be non-negative"
    return result

def relu_derivative(x):
    assert isinstance(x, np.ndarray), "Input to ReLU derivative must be a numpy array"
    result = (x > 0).astype(float)
    assert np.all((result == 0) | (result == 1)), "ReLU derivative must be 0 or 1"
    return result

def softmax(x):
    assert isinstance(x, np.ndarray), "Input to softmax must be a numpy array"
    x_stable = x - np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x_stable)
    result = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    assert np.all((result >= 0) & (result <= 1)), "Softmax output must be in [0, 1]"
    assert np.allclose(np.sum(result, axis=1), 1), "Softmax output must sum to 1 per sample"
    return result
    

###  Data augmentation functions

In [7]:
def rotate_image(image, angle, image_size=(32, 32)):
    """Rotate the image by a given angle."""
    image_2d = image.reshape(image_size)
    center = (image_size[1] // 2, image_size[0] // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image_2d, rotation_matrix, image_size, flags=cv2.INTER_LINEAR)
    return rotated.flatten()

def translate_image(image, tx, ty, image_size=(32, 32)):
    """Translate the image by tx and ty pixels."""
    image_2d = image.reshape(image_size)
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated = cv2.warpAffine(image_2d, translation_matrix, image_size, flags=cv2.INTER_LINEAR)
    return translated.flatten()

def augment_image(image, image_size=(32, 32)):
    """Apply random rotation and translation to an image."""
    # Random rotation between -15 and 15 degrees
    angle = np.random.uniform(-15, 15)
    image = rotate_image(image, angle, image_size)
    
    # Random translation within ±10% of image size
    max_shift = 0.1 * image_size[0]
    tx = np.random.uniform(-max_shift, max_shift)
    ty = np.random.uniform(-max_shift, max_shift)
    image = translate_image(image, tx, ty, image_size)
    
    return image

### MultiClassNeuralNetwork class with Adam optimizer:

In [12]:
class MultiClassNeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.001, l2_lambda=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        assert isinstance(layer_sizes, list) and len(layer_sizes) >= 2, "layer_sizes must be a list with at least 2 elements"
        assert all(isinstance(size, int) and size > 0 for size in layer_sizes), "All layer sizes must be positive integers"
        assert isinstance(learning_rate, (int, float)) and learning_rate > 0, "Learning rate must be a positive number"
        assert isinstance(l2_lambda, (int, float)) and l2_lambda >= 0, "L2 regularization parameter must be non-negative"
        assert isinstance(beta1, (int, float)) and 0 <= beta1 < 1, "Beta1 must be in [0, 1)"
        assert isinstance(beta2, (int, float)) and 0 <= beta2 < 1, "Beta2 must be in [0, 1)"
        assert isinstance(epsilon, (int, float)) and epsilon > 0, "Epsilon must be positive"

        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weights = []
        self.biases = []
        self.m_weights = []
        self.v_weights = []
        self.m_biases = []
        self.v_biases = []
        self.t = 0

        np.random.seed(42)
        for i in range(len(layer_sizes) - 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01
            b = np.zeros((1, layer_sizes[i + 1]))
            assert w.shape == (layer_sizes[i], layer_sizes[i + 1]), f"Weight matrix {i + 1} has incorrect shape"
            assert b.shape == (1, layer_sizes[i + 1]), f"Bias vector {i + 1} has incorrect shape"
            self.weights.append(w)
            self.biases.append(b)
            self.m_weights.append(np.zeros_like(w))
            self.v_weights.append(np.zeros_like(w))
            self.m_biases.append(np.zeros_like(b))
            self.v_biases.append(np.zeros_like(b))

    def forward(self, X):
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"

        self.activations = [X]
        self.z_values = []

        for i in range(len(self.weights) - 1):
            z = self.activations[i] @ self.weights[i] + self.biases[i]
            assert z.shape == (X.shape[0], self.layer_sizes[i + 1]), f"Z^{[{i + 1}]} has incorrect shape"
            self.z_values.append(z)
            self.activations.append(relu(z))

        z = self.activations[-1] @ self.weights[-1] + self.biases[-1]
        assert z.shape == (X.shape[0], self.layer_sizes[-1]), "Output Z has incorrect shape"
        self.z_values.append(z)
        output = softmax(z)
        assert output.shape == (X.shape[0], self.layer_sizes[-1]), "Output A has incorrect shape"
        self.activations.append(output)

        return self.activations[-1]

    def compute_loss(self, y_true, y_pred):
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to loss must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

        m = y_true.shape[0]
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        cross_entropy_loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        
        l2_loss = 0
        for w in self.weights:
            l2_loss += np.sum(np.square(w))
        l2_loss = (self.l2_lambda / (2 * m)) * l2_loss

        total_loss = cross_entropy_loss + l2_loss
        assert not np.isnan(total_loss), "Loss computation resulted in NaN"
        return total_loss

    def compute_accuracy(self, y_true, y_pred):
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to accuracy must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        accuracy = np.mean(predictions == true_labels)
        assert 0 <= accuracy <= 1, "Accuracy must be between 0 and 1"
        return accuracy

    def backward(self, X, y, outputs):
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray) and isinstance(outputs, np.ndarray), "Inputs to backward must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape == outputs.shape, "y and outputs must have the same shape"

        m = X.shape[0]
        self.d_weights = [np.zeros_like(w) for w in self.weights]
        self.d_biases = [np.zeros_like(b) for b in self.biases]
        self.t += 1

        dZ = outputs - y
        assert dZ.shape == outputs.shape, "dZ for output layer has incorrect shape"
        self.d_weights[-1] = (self.activations[-2].T @ dZ) / m
        self.d_biases[-1] = np.sum(dZ, axis=0, keepdims=True) / m

        for i in range(len(self.weights) - 2, -1, -1):
            dZ = (dZ @ self.weights[i + 1].T) * relu_derivative(self.z_values[i])
            assert dZ.shape == (X.shape[0], self.layer_sizes[i + 1]), f"dZ^{[{i + 1}]} has incorrect shape"
            self.d_weights[i] = (self.activations[i].T @ dZ) / m
            self.d_biases[i] = np.sum(dZ, axis=0, keepdims=True) / m

        for i in range(len(self.weights)):
            self.d_weights[i] += self.l2_lambda * self.weights[i] / m

        for i in range(len(self.weights)):
            self.m_weights[i] = self.beta1 * self.m_weights[i] + (1 - self.beta1) * self.d_weights[i]
            self.m_biases[i] = self.beta1 * self.m_biases[i] + (1 - self.beta1) * self.d_biases[i]
            self.v_weights[i] = self.beta2 * self.v_weights[i] + (1 - self.beta2) * (self.d_weights[i] ** 2)
            self.v_biases[i] = self.beta2 * self.v_biases[i] + (1 - self.beta2) * (self.d_biases[i] ** 2)
            m_w_hat = self.m_weights[i] / (1 - self.beta1 ** self.t)
            m_b_hat = self.m_biases[i] / (1 - self.beta1 ** self.t)
            v_w_hat = self.v_weights[i] / (1 - self.beta2 ** self.t)
            v_b_hat = self.v_biases[i] / (1 - self.beta2 ** self.t)
            self.weights[i] -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
            self.biases[i] -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)

    def train(self, X, y, epochs, batch_size, X_val=None, y_val=None):
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray), "X and y must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape[1] == self.layer_sizes[-1], f"Output dimension ({y.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert isinstance(epochs, int) and epochs > 0, "Epochs must be a positive integer"
        assert isinstance(batch_size, int) and batch_size > 0, "Batch size must be a positive integer"

        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []

        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            epoch_loss = 0
            for i in range(0, X.shape[0], batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]

                # Apply data augmentation to the batch
                X_batch_aug = np.array([augment_image(x, image_size=(32, 32)) for x in X_batch])

                outputs = self.forward(X_batch_aug)
                epoch_loss += self.compute_loss(y_batch, outputs)
                self.backward(X_batch_aug, y_batch, outputs)

            train_loss = epoch_loss / (X.shape[0] // batch_size)
            train_pred = self.forward(X)
            train_accuracy = self.compute_accuracy(y, train_pred)
            train_losses.append(train_loss)
            train_accuracies.append(train_accuracy)

            if X_val is not None and y_val is not None:
                val_pred = self.forward(X_val)
                val_loss = self.compute_loss(y_val, val_pred)
                val_accuracy = self.compute_accuracy(y_val, val_pred)
                val_losses.append(val_loss)
                val_accuracies.append(val_accuracy)
            else:
                val_losses.append(np.nan)
                val_accuracies.append(np.nan)

            if epoch % 50 == 0:
                print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_losses[-1]:.4f}, "
                      f"Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracies[-1]:.4f}")

        return train_losses, val_losses, train_accuracies, val_accuracies

    def predict(self, X):
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"

        outputs = self.forward(X)
        predictions = np.argmax(outputs, axis=1)
        assert predictions.shape == (X.shape[0],), "Predictions have incorrect shape"
        return predictions

# Load the data
data_dir = os.path.join(os.getcwd(), './amhcd-data-64/tifinagh-images')
try:
    labels_df = pd.read_csv(os.path.join(data_dir, './amhcd-data-64/labels-map.csv'))
    assert 'image_path' in labels_df.columns and 'label' in labels_df.columns, "CSV must contain 'image_path' and 'label' columns"
except FileNotFoundError:
    print("labels-map.csv not found. Please check the dataset structure.")
    image_paths = []
    labels = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            for img_name in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, img_name))
                labels.append(label_dir)
    labels_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

assert not labels_df.empty, "No data loaded. Check dataset files."
print(f"Loaded {len(labels_df)} samples with {labels_df['label'].nunique()} unique classes.")

# Encode the labels
label_encoder = LabelEncoder()
labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])
num_classes = len(label_encoder.classes_)

# Function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    assert os.path.exists(image_path), f"Image not found: {image_path}"
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to load image: {image_path}"
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    return img.flatten()

# Load all the images
X = np.array([load_and_preprocess_image(os.path.join(data_dir, path)) for path in labels_df['image_path']])
y = labels_df['label_encoded'].values

assert X.shape[0] == y.shape[0], "Mismatch between number of images and labels"
assert X.shape[1] == 32 * 32, f"Expected flattened image size of {32*32}, got {X.shape[1]}"

# One-hot encode the labels
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_one_hot = np.array(one_hot_encoder.fit_transform(y.reshape(-1, 1)))

# Parameters for k-fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)
layer_sizes = [X.shape[1], 64, 32, num_classes]
epochs = 300
batch_size = 32

# Lists to store the results
all_train_losses = []
all_val_losses = []
all_train_accuracies = []
all_val_accuracies = []
all_test_predictions = []
all_test_labels = []

# k-fold loop
fold = 1
for train_index, test_index in kf.split(X):
    print(f"\nTraining on Fold {fold}/{k}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_one_hot[train_index], y_one_hot[test_index]
    
    # Split train into train and validation
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.25, stratify=np.argmax(y_train, axis=1), random_state=42
    )

    # Create and train the model
    nn = MultiClassNeuralNetwork(layer_sizes, learning_rate=0.001, l2_lambda=0.01)
    train_losses, val_losses, train_accuracies, val_accuracies = nn.train(
        X_train, y_train, epochs, batch_size, X_val, y_val
    )

    # Store the metrics
    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)
    all_train_accuracies.append(train_accuracies)
    all_val_accuracies.append(val_accuracies)

    # Predictions on the test set
    y_pred = nn.predict(X_test)
    all_test_predictions.extend(y_pred)
    all_test_labels.extend(np.argmax(y_test, axis=1))

    fold += 1

# Calculate the average metrics
mean_train_losses = np.mean(all_train_losses, axis=0)
mean_val_losses = np.mean(all_val_losses, axis=0)
mean_train_accuracies = np.mean(all_train_accuracies, axis=0)
mean_val_accuracies = np.mean(all_val_accuracies, axis=0)

# Classification report
print("\nRapport de classification (Moyenne sur k folds):")
print(classification_report(all_test_labels, all_test_predictions, target_names=label_encoder.classes_))

labels-map.csv not found. Please check the dataset structure.
Loaded 28182 samples with 33 unique classes.

Training on Fold 1/5
Epoch 0, Train Loss: 3.2744, Val Loss: 3.1064, Train Acc: 0.0934, Val Acc: 0.0905
Epoch 50, Train Loss: 1.2434, Val Loss: 0.8453, Train Acc: 0.7404, Val Acc: 0.7410
Epoch 100, Train Loss: 1.0950, Val Loss: 0.7770, Train Acc: 0.7599, Val Acc: 0.7516
Epoch 150, Train Loss: 1.0403, Val Loss: 0.7450, Train Acc: 0.7619, Val Acc: 0.7547
Epoch 200, Train Loss: 1.0039, Val Loss: 0.6659, Train Acc: 0.7910, Val Acc: 0.7864
Epoch 250, Train Loss: 0.9797, Val Loss: 0.6479, Train Acc: 0.7963, Val Acc: 0.7846

Training on Fold 2/5
Epoch 0, Train Loss: 3.2105, Val Loss: 2.8577, Train Acc: 0.1510, Val Acc: 0.1430
Epoch 50, Train Loss: 1.2998, Val Loss: 0.8511, Train Acc: 0.7380, Val Acc: 0.7291
Epoch 100, Train Loss: 1.1158, Val Loss: 0.7196, Train Acc: 0.7809, Val Acc: 0.7713
Epoch 150, Train Loss: 1.0215, Val Loss: 0.6617, Train Acc: 0.7914, Val Acc: 0.7813
Epoch 200, Trai

In [32]:
# Set plot style for high-quality visualizations
plt.style.use('ggplot')  
plt.rcParams.update({'font.size': 12, 'figure.dpi': 300})

# Plot 1: Confusion Matrix
plt.figure(figsize=(12, 10), dpi=300)
cm = confusion_matrix(all_test_labels, all_test_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'}, 
            annot_kws={"size": 10})
plt.title('Confusion Matrix (Test Set)', fontsize=16, pad=20)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix_high_quality.png', bbox_inches='tight', dpi=300)
plt.show()

In [34]:
# Plot 2: Loss and Precision Curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
# Loss curves
ax1.plot(mean_train_losses, label='Training Loss', color='blue', linewidth=2)
ax1.plot(mean_val_losses, label='Validation Loss', color='orange', linewidth=2)
ax1.set_title('Loss Curves (K-Fold Average)', pad=15)
ax1.set_xlabel('Epoch', labelpad=10)
ax1.set_ylabel('Loss', labelpad=10)
ax1.legend()
ax1.grid(True, linestyle='--', alpha=0.7)

# Precision curves
ax2.plot(mean_train_accuracies, label='Training Accuracy', color='blue', linewidth=2)
ax2.plot(mean_val_accuracies, label='Validation Accuracy', color='orange', linewidth=2)
ax2.set_title('Precision Curves (K-Fold Average)', pad=15)
ax2.set_xlabel('Epoch', labelpad=10)
ax2.set_ylabel('Precision', labelpad=10)
ax2.legend()
ax2.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and save
plt.tight_layout()
plt.savefig('loss_precision_plot_kfold_high_quality.png', dpi=300, bbox_inches='tight')
plt.show()